In [73]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm

from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input

from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout,add

In [ ]:
BASE_DIR = '/kaggle/input/flickr8k'
WORKING_DIR = '/kaggle/working'

## Extract Image Feature

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

# Path to the manually downloaded weights file
weights_path = 'C:\\Users\\hp\\.keras\\models'

# Load VGG16 model with the specified weights file
model = VGG16(weights=None)

# Restructure the model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

# Summary
print(model.summary())


In [ ]:
features={}
directory = os.path.join(BASE_DIR,'Images')

for img_name in tqdm(os.listdir(directory)):
    img_path = directory + '/' + img_name
    image = load_img(img_path , target_size = (224,224))
    
    image = img_to_array(image)
    
    image=image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose =0)
    image_id = img_name.split('.')[0]
    features[image_id] = feature


In [ ]:
pickle.dump(features,open(os.path.join(WORKING_DIR, 'features.pkl'),'wb'))

In [ ]:
with open(os.path.join(WORKING_DIR,'features.pkl'),'rb') as f:
    features=pickle.load(f)

## Load the Captions Data

In [ ]:
with open(os.path.join(BASE_DIR,'captions.txt'),'r') as f:
    next(f)
    captions_doc=f.read()

In [ ]:
mapping = {}
#process
for line  in tqdm(captions_doc.split('\n')):
    tokens=line.split(',')
    if len(line)<2:
        continue
    image_id,caption = tokens[0],tokens[1:]
    image_id = image_id.split('.')[0]
    caption = " ".join(caption)
    if image_id not in mapping:
        mapping[image_id] = []
    mapping[image_id].append(caption)    
        
    

In [ ]:
len(mapping)

## Preprocessing

In [ ]:
def clean (mapping):
    for key,captions in mapping.items():
        for i in range(len(captions)):
            caption=captions[i]
            #convert to lowercase
            caption=caption.lower()
            #delete digits , special characters etc
            caption=caption.replace('[^A-Za-z]','')
            #delete additional spaces
            caption=caption.replace('\s+', ' ')
            #add start and end tags
            caption = '<start>' + " ".join([word for word in caption.split() if len(word)>1]) + '<end>'
            captions[i]=caption
            
            
            
            

In [149]:
mapping['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [ ]:
clean(mapping)

In [150]:
mapping['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [ ]:
all_captions = []
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

In [ ]:
len(all_captions)

In [ ]:
all_captions[:10]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
vocab_size

In [53]:
max_length = max(len(caption.split()) for caption in all_captions)
max_length

37

## Train Test Split

In [58]:
image_ids=list(mapping.keys())
split = int(len(image_ids)*0.90)
train = image_ids[:split]
test=image_ids[split:]

In [118]:
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    n = 0
    X1, X2, y = None, None, []
    
    while True:
        for key in data_keys:
            n += 1
            captions = mapping[key]

            for caption in captions:
                seq = tokenizer.texts_to_sequences([caption])[0]

                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                    if X1 is None:
                        X1 = np.array([features[key][0]])
                    else:
                        X1 = np.concatenate([X1, np.array([features[key][0]])], axis=0)

                    in_seq = np.expand_dims(in_seq, axis=0)

                    if X2 is not None and X2.shape[1] == in_seq.shape[1]:
                        X2 = np.vstack([X2, in_seq])
                    else:
                        X2 = in_seq

                    y.append(out_seq)

            if n == batch_size:
                yield [X1, X2], np.array(y)
                X1, X2, y = None, None, []
                n = 0


## Model Creation

In [ ]:
#save
model.save(WORKING_DIR+'/best_model.h5')

## Generate Captions for the Image

In [ ]:
def idx_to_word(integer,tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer:
            return word
    return none    

In [137]:
def index_to_word(index, tokenizer):
    for word, idx in tokenizer.word_index.items():
        if idx == index:
            return word

In [139]:
def predict_caption(model,image,tokenizer,max_length):
    in_text='<start>'
    for i in range(max_length):
        sequence=tokenizer.texts_to_sequences([in_text])[0]
        sequence=pad_sequences([sequence],max_length)
        yhat=model.predict([image,sequence],verbose=0)
        yhat=np.argmax(yhat)
        word = index_to_word(yhat, tokenizer)
        if word is None:
            break
        in_text += " " + word
        if word == '<end>':
            break
    return in_text       
        
    

In [152]:
from nltk.translate.bleu_score import corpus_bleu
actual,predicted = list(),list()

for key in tqdm(test):
    captions=mapping[key]
    y_pred=predict_caption(model,features[key],tokenizer,max_length)
    actual_captions=[caption.split() for caption in captions]
    y_pred=y_pred.split()
    actual.append(actual_captions)
    predicted.append(y_pred)
    
print("BLEU-1: %f" % corpus_bleu(actual,predicted,weights=(1.0,0,0,0)))
print("BLEU-2: %f" % corpus_bleu(actual,predicted,weights=(0.5,0.5,0,0)))

  0%|          | 0/810 [00:00<?, ?it/s]

BLEU-1: 0.191293
BLEU-2: 0.099497


## Visualization

In [144]:
from PIL import Image
import matplotlib.pyplot as plt
def generate_caption():
    image_name="1007320043_627395c3d8.jpg"
    image_id=image.name.split('.')[0]
    image_path=os.path.join(BASE_DIR,"Images",image_name)
    image=Image.open(img_path)
    captions=mapping[image_id]
    print("-------Actual-----------")
    for caption in caption:
        print(caption)
    y_pred = predict_caption(model,features[image_id],tokenizer,max_length)
    print("-------------Predicted-------------")
    print(y_pred)
    plt.inshow(image)


In [ ]:
generate_caption("1022454332_6af2c1449a.jpg")